# Bernstein-Vazirani alrogithm 

Given function $$f: \{0, 1\}^{n} \rightarrow \{0, 1\}$$
The goal is to determine: $$ f (x) = a \bullet x $$
where $$ a \bullet x = a_0 x_0 \oplus a_1 x_1 \oplus \dots \oplus a_{n_1} x_{n_1} $$

In [1]:
import numpy as np
import quant
from functools import reduce

In [2]:
%load_ext autoreload
%autoreload 2

## Step 1
Create input vector 
$$ | 0 \dots 0 \rangle \oplus | 1 \rangle$$

In [3]:
qubit_number = 3
a = 6

In [4]:
x_0n = reduce(np.kron, np.repeat(quant.ZERO[np.newaxis, :], qubit_number, axis=0))
y1 = quant.ONE
input_vector = np.kron(x_0n, y1)

## Step 2
Apply Hadamard $H^{\oplus n+1}$

In [5]:
H = quant.hadamard_n(qubit_number + 1)

In [6]:
vector_s2 = np.dot(H, input_vector)

## Step 3
Apply function $U_f = | x \rangle | y \oplus f(x) \rangle$


In [7]:
def create_bernstein_vazirani_function(qubit_number, func):
    states = 2 ** qubit_number
    res = np.empty([states, states])
    
    for num in range(states):
        state = format(num, '0' + str(qubit_number) + 'b')
        X = state[:-1]
        y = str(int(state[-1]) ^ func(int(X, 2)))
        column = quant.string_to_state(X+y)
        
        res[:, num] = column[:, 0]
    
    return res

In [8]:
U_f = create_bernstein_vazirani_function(qubit_number + 1, lambda x: quant.bit_count(a & x) & 1)

In [9]:
vector_s3 = np.dot(U_f, vector_s2)

## Step 4
Apply Hadamard to input vectors

In [10]:
H_n = quant.hadamard_n(qubit_number)

In [11]:
vector_s4 = np.dot(np.kron(H_n, np.eye(2)), vector_s3)

## Step 5
Measuring stage

In [12]:
states = 2**qubit_number

for num in range(states):
    basis_str = format(num, '0' + str(qubit_number) + 'b')
    
    basis = quant.string_to_state(basis_str)
    prob = quant.measure(vector_s4, basis)
    print('|' + basis_str + '>', prob)

|000> 0.0
|001> 0.0
|010> 0.0
|011> 0.0
|100> 9.071053470133314e-35
|101> 0.0
|110> 0.9999999999999989
|111> 0.0
